In [ ]:
import polars as pl
import geopandas as gpd
from pathlib import Path
import folium
import pandas as pd
import numpy as np

from matplotlib import pyplot as plt

# Data Load

First, we read in the shapefile for the gridcell covariates.

We will be using EPSG 3174 for the Great Lakes coordinate system, followed by EPSG:4326 for plotting.

See https://epsg.io/3174 and https://epsg.io/4326

try epsg 2163 - clear outputs

In [ ]:
bat_covars = gpd.read_file("data/data/NABat_grid_covariates/NABat_grid_covariates.shp")
stations = pd.read_excel("data/data/USFWS_Bat_Acoustic_Data/Metadata.xlsx", sheet_name='AcousticSites')
pass_counts = pd.read_excel("data/data/USFWS_Bat_Acoustic_Data/NightlyPassCounts.xlsx")
water_bodies = gpd.read_file("data/data/USA_Detailed_Water_Bodies/USA_Detailed_Water_Bodies.shp")
lakes = gpd.read_file("data/data/rivers_and_lakes_shapefile/NA_Lakes_and_Rivers/data/lakes_p/northamerica_lakes_cec_2023.shp")
rivers = gpd.read_file("data/data/rivers_and_lakes_shapefile/NA_Lakes_and_Rivers/data/rivers_l/northamerica_rivers_cec_2023.shp")


In [ ]:
lakes = lakes.to_crs(epsg=2163)
rivers = rivers.to_crs(epsg=2163)
lakes.plot()
rivers.plot()

In [ ]:
water_bodies = water_bodies.to_crs(epsg=2163)
water_bodies.head()
# water_bodies = water_bodies[water_bodies["SQKM"] > 1.0]
# all_water_bodies = gpd.GeoDataFrame(all_water_bodies, geometry='geometry')

water_bodies.plot()

In [ ]:
all_water_bodies = gpd.GeoDataFrame(pd.concat([
    water_bodies,
    lakes,
    rivers
], axis=0, ignore_index=True), geometry='geometry')

all_water_bodies["NAME"] = all_water_bodies["NAME"].combine_first(all_water_bodies["NameEn"]).fillna("Unknown")
all_water_bodies.head()


In [ ]:
stations["geometry"] = gpd.points_from_xy(stations["Long"], stations["Lat"], crs="EPSG:4326").to_crs(epsg=2163)
stations = gpd.GeoDataFrame(stations, geometry='geometry')

Now, merge the two datasets together.

In [ ]:
station_plus_water = (
    gpd.sjoin_nearest(
        stations, all_water_bodies, how="left", distance_col="dist_to_water", rsuffix="_water"
    )
    .sort_values("dist_to_water", ascending=False)
    .groupby("AcousticSite")
    .nth(0)
)

nearest_water = all_water_bodies.loc[station_plus_water["index__water"]]

# station_plus_water = station_plus_water.set_geometry("geometry")

station_plus_water = station_plus_water.to_crs("EPSG:4326")
station_plus_water.plot()

water_geom = nearest_water[["NAME", "geometry"]].to_crs("EPSG:4326")
water_geom.plot()


In [ ]:
water_geom["NAME"]

In [ ]:
m = folium.Map(zoom_start=4, location=[38.5, -90.0])

plotted_geoms = set()
i = 0
for (row, water_geo) in zip(station_plus_water.iterrows(), water_geom.iterrows()):

    # print(i)
    _, r = row
    _, w = water_geo
    poly_dat = gpd.GeoSeries(r["geometry"]).to_json()

    water_dat = gpd.GeoSeries(w["geometry"]).to_json()
    geo_j = folium.GeoJson(data=poly_dat)
    water_j = folium.GeoJson(data=water_dat)
    info = pd.DataFrame(
        r[["AcousticSite", 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, "dist_to_water", "NAME"]],
    ).to_html()

    w_info = w["NAME"]

    folium.Popup(info).add_to(geo_j)
    folium.Popup(w_info).add_to(water_j)

    geo_j.add_to(m)
    if water_dat not in plotted_geoms:
        water_j.add_to(m)
        plotted_geoms.add(water_dat)

    i += 1
    if i % 50 == 0:
        print(f"{i} ({i/len(station_plus_water)*100:.0f}%)")
    

m

In [ ]:
m.save("bat_maps_distance.html")
